In [1]:
import pytest
import random
import math
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [28]:
def GenerateData(select, CL1=7, CL2=14, CL3=21, seed=None):
    if seed:
        random.seed(100)
    if select == 1:
        data = np.zeros((CL2,2))
        for i in range(0,CL1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1,CL2):
            data[i,0] = 3 + random.random()
            data[i,1] = 3 + random.random()
    elif select == 2:
        data = np.zeros((CL3,2))
        for i in range(0,CL1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1,CL2):
            data[i,0] = 3 + random.random()
            data[i,1] = 1 + random.random()

        for i in range(CL2,CL3):
            data[i,0] = 1.5 + random.random()
            data[i,1] = 3 + random.random()
    return data

def CalculateAffinity(data):
    sigma = 1
    col_size = data.shape[0]
    dim = data.shape[1]
    affinity = np.zeros((col_size,col_size))

    for i in range(col_size):
        for j in range(col_size):
            dist_tmp = 0
            for k in range(dim):
                dist_tmp +=(data[i,k]-data[j,k])**2
            dist = math.sqrt(dist_tmp)
            affinity[i,j] = math.exp(-dist/(2*sigma**2))
    return affinity

def Spectral(k,affinity):
  
    # compute the degree matrix
    D =  np.zeros((affinity.shape))
    for i in range(affinity.shape[0]):
        D[i,i] = sum(affinity[i,:])


    #compute the normalized laplacian / affinity matrix (method 1)
    NL1 = np.zeros((affinity.shape))
    for i in range(affinity.shape[0]):
         for j in range(affinity.shape[1]):
                NL1[i,j] = affinity[i,j]/(math.sqrt(D[i,i]*math.sqrt(D[j,j])))

    eigvalue,eigvector = la.eig(NL1)
    sort_index = np.argsort(eigvalue)
    new_eigvec = eigvector[:,sort_index[-k:][::-1]]

    
    #new_eigvec = eigvector[:,(eigvector.shape[0]-(k-1)):(eigvector.shape[0]+1)]
    #construct the normalized matrix U from the obtained eigen vectors
    U = np.zeros((new_eigvec.shape))
    for i in range(new_eigvec.shape[0]):
        n = math.sqrt(sum(new_eigvec[i,:]**2));   
        U[i,:] = new_eigvec[i,:]/n 
    return U

In [48]:
withSeed = np.array([[ 1.14566926,  1.454927  ],
       [ 1.77078381,  1.70551323],
       [ 1.73195897,  1.43351443],
       [ 1.80002046,  1.53290141],
       [ 1.08015371,  1.45594588],
       [ 1.04788752,  1.9329624 ],
       [ 1.94707801,  1.33535078],
       [ 3.30940593,  1.76801815],
       [ 3.20386953,  1.17846076],
       [ 3.18859491,  1.34700445],
       [ 3.62632164,  1.96331578],
       [ 3.21083399,  1.95610065],
       [ 3.55539967,  1.90115204],
       [ 3.81801819,  1.16042181],
       [ 2.14854291,  3.12409328],
       [ 1.50564509,  3.39551618],
       [ 2.2735007 ,  3.56615771],
       [ 1.69264066,  3.84117471],
       [ 2.41377684,  3.23722506],
       [ 1.94819248,  3.63779981],
       [ 2.40234302,  3.12661521]])

In [49]:
"""Generate Data"""
data1 = GenerateData(2,seed=True)
assert data1.shape == (21,2)
assert np.allclose(data1, withSeed)
assert type(data1) == np.ndarray

In [55]:
withSeedAffinity0 = np.array([ 1.        ,  0.7140981 ,  0.74576831,  0.71929024,  0.96776912,
         0.78351368,  0.66688328,  0.33516388,  0.35404096,  0.35955535,
         0.28192737,  0.34557224,  0.29365529,  0.26073155,  0.37770703,
         0.37275008,  0.30216027,  0.29403203,  0.33497615,  0.31259077,
         0.35145343])

In [56]:
"""Calculate Affinity"""
affinity = CalculateAffinity(data1)
assert affinity.shape == (21,21)
assert np.allclose(affinity[0],withSeedAffinity0)
assert type(affinity) == np.ndarray

In [60]:
withSeedSpectral0 = np.array([ 0.5048797 ,  0.02696841,  0.86276833])

In [61]:
"""Spectral"""
spectral = Spectral(3, affinity)
assert spectral.shape == (21,3)
assert np.allclose(spectral[0],withSeedSpectral0)
assert type(affinity) == np.ndarray